In [12]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from HorizontalMaxPool2d import HorizontalMaxPool2d
from IPython import embed 

In [44]:
class ReIDNet(nn.Module):
    def __init__(self, num_classes, loss={'softmax, metric'}, aligned=False,**kwargs):
        super().__init__()
        orinet=torchvision.models.alexnet(pretrained=True)
        #print(orinet)
        self.loss=loss
        self.basenet=nn.Sequential(*list(orinet.children())[0:1])
        #print(*list(orinet.children()))
        #print(self.basenet)
        self.linear=nn.Linear(256, 2048)
        self.classifier=nn.Linear(2048, num_classes)
        self.aligned=aligned
        if self.aligned:
            self.horizon_pool=HorizontalMaxPool2d()
            self.bn=nn.BatchNorm2d(256)
            self.relu=nn.ReLU(inplace=True)
            self.conv1=nn.Conv2d(256,128,kernel_size=1,stride=1,padding=0,bias=True)
    def forward(self, x):
        x=self.basenet(x)
        cnn_result=F.avg_pool2d(x, x.size()[2:])
        cnn_result=cnn_result.view(x.size(0),-1)
        f=self.linear(cnn_result)
        #归一化
        f=1.*f / (torch.norm(f,2,dim=-1, keepdim=True).expand_as(f)+1e-12)
        #如果是用于测试，只返回feature张量
        if not self.training:
            return f
        #如果使用Alinged分支，则加以计算local feature；如果不使用Alinged分支，则根据度量学习与表征学习分类
        if self.aligned:
            print(x.size())
            lf=self.bn(x)
            print('1:',lf.size())
            lf=self.relu(lf)
            print('2:',lf.size())
            lf=self.horizon_pool(lf)
            print('3:',lf.size())
            lf=self.conv1(lf)
            print('4:',lf.size())
            y=self.classifier(f)
            return y,f,lf
        else:
            if self.loss=={'softmax'}:
                y=self.classifier(f)
                return y
            elif self.loss=={'metric'}:
                return f
            elif self.loss=={'softmax, metric'}:
                y=self.classifier(f)
                return y,f
            else:
                print('loss setting error')

In [45]:
if __name__=='__main__':
    model=ReIDNet(num_classes=751,aligned=True)
    imgs=torch.rand(32,3,128,64)
    y,f,lf=model(imgs)
    print(y.size())
    print(f.size()) 
    print(lf.size())

torch.Size([32, 256, 3, 1])
1: torch.Size([32, 256, 3, 1])
2: torch.Size([32, 256, 3, 1])
3: torch.Size([32, 256, 3, 1])
4: torch.Size([32, 128, 3, 1])
torch.Size([32, 751])
torch.Size([32, 2048])
torch.Size([32, 128, 3, 1])
